# Merging some data

The datasets being merged only applies to california but if you can find some other datasets that have the name of the cities as a column it should not be too difficult since python and pandas provide convenient ways to work with [text](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html). 

In this notebook i will be merging the `latitude` and `longitude` variables from the kaggle dataset cal_cities_lat_long.csv. A `population` variable for each city after scraping it from towncharts.com demographics web page for each city. And finally, `area_total_km2` variable from Jake Vanderplas hosted california_cities.csv dat. Making it 4 new variable being merged funny enough i notice these last dataset had lat and long

In [1]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle


In [13]:
cities_data=pd.read_csv("C:\\Users\\Crist\\Towncharts\\California_Housing_Project\\datasets\\cal_cities_lat_long.csv")

In [2]:
housing = pd.read_csv("C:\\Users\\Crist\\Towncharts\\California_Housing_Project//datasets/Towncharts_California_Housing.csv",index_col =0)

In [10]:
len(housing)

453

In [27]:
cities_data.loc[cities_data.Name.str.contains('Monterey')] 

,Name,Latitude,Longitude
256,Monterey,36.600239,-121.894675
257,Monterey Park,34.062511,-118.122847


In [22]:
housing[housing.city_name.str.contains('Monterey')]

,city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,...,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999
251,Monterey,13371.0,1541.5,-1.6,63.2,36.8,696200.0,1625.0,1885.0,22.0,...,0.098,0.568,0.229,0.137,0.258,0.693,0.030,0.098,0.846,0.008
252,Monterey Park,21336.0,2781.9,2.3,46.3,53.7,566700.0,1275.0,1484.0,20.0,...,0.405,0.404,0.033,0.289,0.629,0.192,0.084,0.124,0.672,0.022


In [28]:
cities_data.loc[cities_data.Name.str.contains('San Fran')] 

,Name,Latitude,Longitude
351,San Francisco,37.774931,-122.419417
400,South San Francisco,37.654656,-122.407750


In [26]:
housing[housing.city_name.str.contains('San Francisco')]

,city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,...,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999
345,San Francisco,390376.0,8324.6,3.6,62.7,37.3,927400.0,1709.0,2435.0,21.0,...,0.114,0.193,0.572,0.075,0.161,0.662,0.059,0.112,0.671,0.06
394,South San Francisco city-CA,21856.0,2381.5,0.2,38.9,61.1,697400.0,1751.0,2166.0,22.0,...,0.125,0.351,0.439,0.078,0.222,0.683,0.049,0.044,0.825,0.01


Merging the `latitude` and `longitude`



The following lines of code will create a new city name column in our housing DataFrame that will be used to merge the appropriate latitude an longtitude to the correct row of the housing DataFrame. As currently our housing has the name of the city formatted as Alameda city-CA and the cal_cities_lat_long.csv  has the name formatted as Alemeda. 

In [45]:
mytext = list(housing.city_name.copy())
housing['city'] = [(i.split('city',1)[0]).rstrip()  for i in mytext]


In [46]:
california_housing = housing.merge(cities_data, left_on='city',right_on='Name')

In [47]:
california_housing = california_housing.drop(columns = ['city_name','Name'])

Merging the `population` variable

In [ ]:
population_dat.to_csv('C:\\Users\\Crist\\Towncharts\\California_Housing_Project\\datasets\\population_dat.csv')

In [62]:
population_dat = pd.read_csv("C:\\Users\\Crist\\Towncharts\\California_Housing_Project\\datasets\\population_dat.csv",index_col=0)

In [72]:
california_housing = california_housing.merge(population_dat, left_on='city',right_on='city')

Merging `area_total_sq_mi`

In [79]:
area_dat = (pd.read_csv('https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/california_cities.csv',index_col=0)).loc[:,['city','area_total_sq_mi']]

The dataset from jake vanderplas has city name with no spaces
so a quick replace should do the job so we are able to merge. Also some cities have missing `area_total_sq_mi`

In [86]:
california_housing['city'] = california_housing['city'].str.replace(" ","")

In [87]:
california_housing = california_housing.merge(area_dat,left_on='city',right_on='city',how='left')

In [90]:
california_housing.to_csv('C:\\Users\\Crist\\Towncharts\\California_Housing_Project\\datasets\\california_housing.csv')

#  The End